--- Day 8: Treetop Tree House ---

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a [tree house](https://en.wikipedia.org/wiki/Tree_house).

First, determine whether there is enough tree cover here to keep a tree house **hidden**. To do this, you need to count the number of trees that are **visible from outside the grid** when looking directly along a row or column.

The Elves have already launched a [quadcopter](https://en.wikipedia.org/wiki/Quadcopter) to generate a map with the height of each tree (your puzzle input). For example:
```
30373
25512
65332
33549
35390
```
Each tree is represented as a single digit whose value is its height, where `0` is the shortest and `9` is the tallest.

A tree is **visible** if all of the other trees between it and an edge of the grid are **shorter** than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the **interior nine trees** to consider:

- The top-left `5` is **visible** from the left and top. (It isn't visible from the right or bottom since other trees of height `5` are in the way.)
- The top-middle `5` is **visible** from the top and right.
- The top-right `1` is not visible from any direction; for it to be visible, there would need to only be trees of height **`0`** between it and an edge.
- The left-middle `5` is **visible**, but only from the right.
- The center `3` is not visible from any direction; for it to be visible, there would need to be only trees of at most height `2` between it and an edge.
- The right-middle `3` is **visible** from the right.
- In the bottom row, the middle `5` is **visible**, but the `3` and `4` are not.
With 16 trees visible on the edge and another 5 visible in the interior, a total of `21` trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

In [1]:
# Data Load
example_lines = [
    "30373",
    "25512",
    "65332",
    "33549",
    "35390",
]

with open("day_8_input.txt") as _file:
    data_lines = _file.readlines()

In [2]:
# Common Code
from __future__ import annotations
import typing
from functools import reduce

class Tree:
    class _Direction:
        UP = "up"
        DOWN = "down"
        LEFT = "left"
        RIGHT = "right"

        @classmethod
        def all(cls):
            return cls.UP, cls.DOWN, cls.LEFT, cls.RIGHT

    height: int
    left: typing.Optional[Tree]
    right: typing.Optional[Tree]
    up: typing.Optional[Tree]
    down: typing.Optional[Tree]

    def __init__(self, height: int):
        self.height = height
        self.left = None
        self.right = None
        self.up = None
        self.down = None
        self._max_heights = {}
        self._trees_seen = {}

    def _get_next_tree(self, direction: str) -> typing.Optional[Tree]:
        if direction == self._Direction.LEFT:
            return self.left
        elif direction == self._Direction.RIGHT:
            return self.right
        elif direction == self._Direction.UP:
            return self.up
        elif direction == self._Direction.DOWN:
            return self.down

        raise RuntimeError(f"Unexpected direction: {direction}")

    def _get_max_height_from(self, direction: str):
        if direction not in self._max_heights:
            max_height = -1

            next_tree = self._get_next_tree(direction)
            if next_tree:
                max_height = max(next_tree._get_max_height_from(direction), next_tree.height)

            self._max_heights[direction] = max_height

        return self._max_heights[direction]

    def _is_visible_from(self, direction):
        return self.height > self._get_max_height_from(direction)

    def is_visible_from_outside(self):
        return any(self._is_visible_from(x) for x in self._Direction.all())
    
    def _trees_seen_from(self, direction):
        if direction not in self._trees_seen:
            next_tree = self._get_next_tree(direction)
            count = 0
            while next_tree:
                count += 1
                if next_tree.height >= self.height:
                    break
                next_tree = next_tree._get_next_tree(direction)

            self._trees_seen[direction] = count

        return self._trees_seen[direction]
    
    def get_scenic_score(self):
        return reduce(lambda x, y: x * y, (self._trees_seen_from(direction) for direction in self._Direction.all()))


class TreeGrid:
    def __init__(self, grid: typing.Iterable[typing.Iterable[Tree]]):
        self._grid = grid

    @staticmethod
    def parse(lines):
        grid = [[Tree(int(height)) for height in line.strip()] for line in lines]

        up_bound = 0
        down_bound = len(grid) - 1
        for row_index, row in enumerate(grid):
            left_bound = 0
            right_bound = len(row) - 1
            for col_index, tree in enumerate(row):
                if col_index > left_bound:
                    tree.left = grid[row_index][col_index - 1]
                if col_index < right_bound:
                    tree.right = grid[row_index][col_index + 1]
                if row_index > up_bound:
                    tree.up = grid[row_index - 1][col_index]
                if row_index < down_bound:
                    tree.down = grid[row_index + 1][col_index]

        return TreeGrid(grid)

    @property
    def visible_tree_count(self):
        return sum(x.is_visible_from_outside() for row in self._grid for x in row)

    def print_heights(self):
        for row in self._grid:
            print("".join(str(x.height) for x in row))

    def print_visible_from_outside(self):
        for row in self._grid:
            print("".join("X" if x.is_visible_from_outside() else "_" for x in row))
            
    @property
    def max_scenic_score(self):
        return max(x.get_scenic_score() for row in self._grid for x in row)
            
    def print_scenic_scores(self):
        for i, row in enumerate(self._grid):
            print(",".join(str(x.get_scenic_score()) for x in row))

In [3]:
# Solution 1 Code
def solution_1(lines, display):
    grid = TreeGrid.parse(lines)
    if display:
        grid.print_heights()
        print()
        grid.print_visible_from_outside()
        print()
    return grid.visible_tree_count


def test_solution_1():
    actual = solution_1(example_lines, True)
    expected = 21

    print(actual, expected)
    assert actual == expected
test_solution_1()

30373
25512
65332
33549
35390

XXXXX
XXX_X
XX_XX
X_X_X
XXXXX

21 21


In [4]:
# Solution 1
solution_1(data_lines, True)

133120320210233440424211425033311533112110111336536142004454550513525522325223123404213204010312200
201131111014211324423255354022022243226445013613610423653614522135534505055120330313403031333103010
312033232323231244025301315245341424106564334260061464515142114141551050411122254121204214442432210
101223224313414001513045124413405604251532415616200623342234245060512030200002055453441222043022132
302000022010423415300513533001221654231646352603366222420353100303160621313144421120541324031322221
022041343104313414435021243154525050055065506466215004364316065443031303052541012204020300331224312
103341401201343234101335152026143021665443260024145323023321363412215203352344024010350241320230403
113420142141151035404302242616255522460104100534576523546245441245051223305614204355454223220120301
141313120334140413444311032422155464031057323373712672513175353623531154646460010504312302300002224
222122233241135333145102102244620325637577115357373355166254434732406430504542404102101335430322111


1703

--- Part Two ---

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of **trees**.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large [eaves](https://en.wikipedia.org/wiki/Eaves) to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:
```
30373
25512
65332
33549
35390
```
- Looking up, its view is not blocked; it can see **`1`** tree (of height `3`).
- Looking left, its view is blocked immediately; it can see only **`1`** tree (of height `5`, right next to it).
- Looking right, its view is not blocked; it can see **`2`** trees.
- Looking down, its view is blocked eventually; it can see **`2`** trees (one of height `3`, then the tree of height `5` that blocks its view).
A tree's **scenic score** is found by **multiplying together** its viewing distance in each of the four directions. For this tree, this is **`4`** (found by multiplying `1 * 1 * 2 * 2`).

However, you can do even better: consider the tree of height `5` in the middle of the fourth row:
```
30373
25512
65332
33549
35390
```
- Looking up, its view is blocked at **`2`** trees (by another tree with a height of `5`).
- Looking left, its view is not blocked; it can see **`2`** trees.
- Looking down, its view is also not blocked; it can see **`1`** tree.
- Looking right, its view is blocked at **`2`** trees (by a massive tree of height `9`).
This tree's scenic score is **`8`** (`2 * 2 * 1 * 2`); this is the ideal spot for the tree house.

Consider each tree on your map. **What is the highest scenic score possible for any tree?**

In [5]:
# Solution 2 Code
def solution_2(lines, display):
    grid = TreeGrid.parse(lines)
    if display:
        grid.print_scenic_scores()
        print()
    return grid.max_scenic_score

def test_solution_2():
    actual = solution_2(example_lines, True)
    expected = 8
    
    print(actual, expected)
    assert actual == expected
test_solution_2()

0,0,0,0,0
0,1,4,1,0
0,6,1,2,0
0,1,8,3,0
0,0,0,0,0

8 8


In [6]:
# Solution 2
solution_2(data_lines, False)

496650